In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sentencebert_missingpositionprediction import *

I0529 09:06:22.392773 139668468299584 file_utils.py:41] PyTorch version 1.4.0 available.


In [3]:
import os
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
device = torch.device('cuda')

In [5]:
# train
train_dataset = ROCStoriesDataset_random_missing(data_path = "../data/rocstories_completion_train.csv")

# dev
val_dataset = ROCStoriesDataset_with_missing(data_path = "../data/rocstories_completion_dev.csv")

# test
test_dataset = ROCStoriesDataset_with_missing(data_path = "../data/rocstories_completion_test.csv")

In [6]:
%%capture

batch_size = 32
block_size = 32

sentbertmodel = SentenceTransformer('bert-base-nli-mean-tokens')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# --- model --- 
model = MissingPisitionPredictionModel(sentbertmodel, device).to(device)

# --- DataLoader ---|
collate_fn = lambda data: collate(data, tokenizer, block_size=block_size, device=device)
train_iterator = DataLoader(
    train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    
valid_iterator = DataLoader(
    val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    
test_iterator = DataLoader(
    test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    

TRG_PAD_IDX = tokenizer.pad_token_id
START_ID = tokenizer.cls_token_id
mpe_criterion = nn.CrossEntropyLoss()

## load model and get accuracy

In [9]:
accuracy_list = {}

In [10]:
model_dirs = ["<PATH to trained model directories>"]
model_name = "sentbert-positionestimation_model.pt"

In [15]:
for model_dir in model_dirs:
    print(os.path.basename(os.path.dirname(model_dir)))

20191228_trial08_30epochs_seed42
20191228_trial09_30epochs_seed1234
20191228_trial10_30epochs_seed99
20191228_trial11_30epochs_seed613
20191228_trial12_30epochs_seed1000


In [16]:
for model_dir in model_dirs:
    print(os.path.basename(os.path.dirname(model_dir)))

    model_path = os.path.join(model_dir, model_name)
    model = MissingPsitionPredictionModel(sentbertmodel, device).to(device)
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = evaluate(model, test_iterator, mpe_criterion)
    
    accuracy_list[os.path.basename(os.path.dirname(model_dir))] = test_acc

20191228_trial08_30epochs_seed42


Iteration: 100%|██████████| 307/307 [02:33<00:00,  2.00it/s]


20191228_trial09_30epochs_seed1234


Iteration: 100%|██████████| 307/307 [04:03<00:00,  1.26it/s]


20191228_trial10_30epochs_seed99


Iteration: 100%|██████████| 307/307 [04:35<00:00,  1.11it/s]


20191228_trial11_30epochs_seed613


Iteration: 100%|██████████| 307/307 [04:34<00:00,  1.12it/s]


20191228_trial12_30epochs_seed1000


Iteration: 100%|██████████| 307/307 [04:35<00:00,  1.12it/s]


In [17]:
accuracy_list

{'20191228_trial08_30epochs_seed42': 0.5229682410423453,
 '20191228_trial09_30epochs_seed1234': 0.5232451140065146,
 '20191228_trial10_30epochs_seed99': 0.5183021172638437,
 '20191228_trial11_30epochs_seed613': 0.5205700325732899,
 '20191228_trial12_30epochs_seed1000': 0.5233184039087948}

In [18]:
import json    

In [19]:
with open("./result/mpp_gru_test_accuracy.json", "w") as f:
    json.dump(accuracy_list, f, indent=4)

# mean, std

In [20]:
import statistics as st

In [21]:
values = np.array(list(accuracy_list.values()))

In [22]:
values = values * 100

In [23]:
values

array([52.2968241 , 52.3245114 , 51.83021173, 52.05700326, 52.33184039])

In [24]:
print('statistics')

print("mean")
print(st.mean(values))

print("std")
print(st.stdev(values))

statistics
mean
52.168078175895765
std
0.22042930055218343
